In [1]:
from pyflink.table import TableEnvironment, EnvironmentSettings

env_settings = EnvironmentSettings.in_streaming_mode()
t_env = TableEnvironment.create(env_settings)

# specify connector and format jars
t_env.get_config().set("pipeline.jars", "file:///opt/flink-1.15.0/lib/flink-sql-connector-kafka-1.15.0.jar;file:///opt/flink-1.15.0/lib/flink-sql-avro-confluent-registry-1.15.0.jar;file:///opt/flink-1.15.0/lib/flink-sql-avro-1.15.0.jar")
    

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [2]:
# wget -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-sql-avro-confluent-registry/1.15.0/flink-sql-avro-confluent-registry-1.15.0.jar
# wget -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-sql-avro/1.15.0/flink-sql-avro-1.15.0.jar

# kafka-avro-console-producer --broker-list localhost:9092 --topic products   --property value.schema='{"type":"record","name":"product","fields":[{"name":"name","type":"string"},{"name":"price","type":"int"}]}'

# {"name":"iphone", "price" : 100}

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic products_sink_topic
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  sink_topic  --from-beginning  


In [3]:
source_ddl = """
       CREATE TABLE products (
        name STRING,
        price INTEGER
        ) WITH (
            'connector' = 'kafka',
            'topic' = 'products',
            'properties.bootstrap.servers' = 'localhost:9092',
            'properties.group.id' = 'testGroup3',
            'scan.startup.mode' = 'latest-offset',
            'properties.auto.offset.reset' = 'latest',
            'value.format' = 'avro-confluent',
            'value.avro-confluent.schema-registry.url' = 'http://localhost:8081',
            'value.fields-include' = 'EXCEPT_KEY'
        )
"""


In [4]:
sink_ddl = """
        CREATE TABLE sink_table(
            name VARCHAR,
            price INTEGER
        ) WITH (
          'connector' = 'kafka',
          'topic' = 'products_sink_topic',
          'properties.bootstrap.servers' = 'localhost:9092',
          'format' = 'json'
        )
        """

In [ ]:
t_env.execute_sql(source_ddl).print()
t_env.execute_sql(sink_ddl).print()
# t_env.execute_sql('select * from user_behavior3').print()

sink = t_env.sql_query("SELECT a FROM products")
sink.execute_insert("sink_table").wait()

OK
OK
